# Bài toán:
Xác định nhãn của các bài báo văn bản Tiếng Việt

In [1]:
from pyvi import ViTokenizer, ViPosTagger # thư viện NLP tiếng Việt
from tqdm import tqdm
import numpy as np
import gensim # thư viện NLP
import json
import os
# import warnings filter
from warnings import simplefilter
# ignore all future warnings
simplefilter(action='ignore', category=FutureWarning)

In [2]:
#Stop-words
from string import punctuation
stop_word = []
with open(r"C:\Users\ASUS ZenBook\OneDrive\Desktop\vietnamese-stopwords-master\vietnamese-stopwords.txt",encoding="utf-8") as f :
    text = f.read()
    for word in text.split() :
        stop_word.append(word)
    f.close()

In [3]:
punc = list(punctuation)
stop_word = stop_word + punc
print(stop_word)

['a', 'lô', 'a', 'ha', 'ai', 'ai', 'ai', 'ai', 'nấy', 'ai', 'đó', 'alô', 'amen', 'anh', 'anh', 'ấy', 'ba', 'ba', 'ba', 'ba', 'bản', 'ba', 'cùng', 'ba', 'họ', 'ba', 'ngày', 'ba', 'ngôi', 'ba', 'tăng', 'bao', 'giờ', 'bao', 'lâu', 'bao', 'nhiêu', 'bao', 'nả', 'bay', 'biến', 'biết', 'biết', 'bao', 'biết', 'bao', 'nhiêu', 'biết', 'chắc', 'biết', 'chừng', 'nào', 'biết', 'mình', 'biết', 'mấy', 'biết', 'thế', 'biết', 'trước', 'biết', 'việc', 'biết', 'đâu', 'biết', 'đâu', 'chừng', 'biết', 'đâu', 'đấy', 'biết', 'được', 'buổi', 'buổi', 'làm', 'buổi', 'mới', 'buổi', 'ngày', 'buổi', 'sớm', 'bà', 'bà', 'ấy', 'bài', 'bài', 'bác', 'bài', 'bỏ', 'bài', 'cái', 'bác', 'bán', 'bán', 'cấp', 'bán', 'dạ', 'bán', 'thế', 'bây', 'bẩy', 'bây', 'chừ', 'bây', 'giờ', 'bây', 'nhiêu', 'bèn', 'béng', 'bên', 'bên', 'bị', 'bên', 'có', 'bên', 'cạnh', 'bông', 'bước', 'bước', 'khỏi', 'bước', 'tới', 'bước', 'đi', 'bạn', 'bản', 'bản', 'bộ', 'bản', 'riêng', 'bản', 'thân', 'bản', 'ý', 'bất', 'chợt', 'bất', 'cứ', 'bất', 'giác', 

# Chuẩn bị

Ta sử dụng hàm get_label để xử lý xâu tên file json về nhãn. Dữ liệu có tổng cộng 45 nhãn được liệt kê trong mảng label. Các dữ liệu phân ra từng nhãn không đều nhau sẽ khiến cho mô hình phân loại bị sai lệch kết quả. Ta sẽ bỏ bớt các văn bản và nhãn mà trong đó số dữ liệu được phân loại vào đó nhỏ hơn 1000 và chỉ lấy những văn bản có nhãn xuất hiện nhiều nhất. Cụ thể là 32 nhãn đã được lấy ra và cho vào mảng m.

In [4]:
label=['an ninh', 'an ninh trat tu', 'bat dong san', 'bien dao',
       'buu chinh vien thong', 'chinh sach', 'chinh tri', 'cong nghiep',
       'dan toc', 'dau thau', 'do thi', 'doi ngoai', 'du lich',
       'giai tri', 'giao duc', 'giao thong van tai', 'hai quan',
       'khoa hoc cong nghe', 'kiem toan', 'kinh doanh', 'kinh te',
       'lao dong', 'lao dong thuong binh va xa hoi', 'nang luong',
       'nong nghiep', 'phap luat', 'phu nu', 'quoc hoi', 'quoc phong',
       'so huu tri tue', 'suc khoe', 'tai chinh', 'tai nguyen moi truong',
       'thanh nien', 'thanh tra', 'the thao', 'thoi trang',
       'thong tin va truyen thong', 'thuong mai', 'tre em', 'van hoa',
       'van nghe', 'xa hoi', 'xay dung', 'y te']
tmp=[]
for i in ([33, 34, 18, 3, 28, 7, 5, 27, 4, 20, 31, 36, 40, 43, 12, 35, 25, 16, 42, 14, 13, 30, 19, 2, 15, 44, 6, 32, 26, 17, 38, 41]):
    tmp.append(label[i])
print(tmp)


m = ['thanh nien', 'thanh tra', 'kiem toan', 'bien dao',
                'quoc phong', 'cong nghiep', 'chinh sach', 'quoc hoi', 
                'buu chinh vien thong', 'kinh te', 'tai chinh', 'thoi trang', 
                'van hoa', 'xay dung', 'du lich', 'the thao', 'phap luat', 
                'hai quan', 'xa hoi', 'giao duc', 'giai tri', 'suc khoe', 
                'kinh doanh', 'bat dong san', 'giao thong van tai', 'y te', 
                'chinh tri', 'tai nguyen moi truong', 'phu nu', 
                'khoa hoc cong nghe', 'thuong mai', 'van nghe']

def get_label(s):   # chuyển tên file thành label
    s=s.replace('.json','')
    for i in range(10):
        s=s.replace(str(i),'')    
    s=s.split('_')
    s = ' '.join(s)
    s=s.replace('  ','')
    if s[len(s)-1]==' ':
        s=s[:len(s)-1]
    return s

['thanh nien', 'thanh tra', 'kiem toan', 'bien dao', 'quoc phong', 'cong nghiep', 'chinh sach', 'quoc hoi', 'buu chinh vien thong', 'kinh te', 'tai chinh', 'thoi trang', 'van hoa', 'xay dung', 'du lich', 'the thao', 'phap luat', 'hai quan', 'xa hoi', 'giao duc', 'giai tri', 'suc khoe', 'kinh doanh', 'bat dong san', 'giao thong van tai', 'y te', 'chinh tri', 'tai nguyen moi truong', 'phu nu', 'khoa hoc cong nghe', 'thuong mai', 'van nghe']


# Nhập dữ liệu

Trong quá trình nhập các bộ X,y (văn bản và nhãn tương ứng) ta sẽ loại ra các văn bản rỗng (nhiễu) và chỉ lấy các văn bản có nhãn xuất hiện nhiều nằm trong mảng m. Đồng thời, ta sẽ lọc những stop-words ra khỏi văn bản.

In [5]:
X=[]
y=[]

folder_path ="C:/Users/ASUS ZenBook/OneDrive/Desktop/json"
def get_label1(s):   # chuyển tên file thành label
    s=s.replace('.json','')
    
    for i in range(10):
        s=s.replace(str(i),'')
    
    s=s.split('_')
    s = ' '.join(s)
    s=s.replace('  ','')
    if s[len(s)-1]==' ':
        s=s[:len(s)-1]
    return s

dirs = os.listdir(folder_path)
for path in (dirs):
    with open (os.path.join(folder_path, path), encoding = "utf8",) as f:
        data = json.load(f)
        print(path)
        for i in data["response"]["docs"]:
            k=i["message"]
            if k!= '':
                k=get_label1(path)
                if k in m:
                    k=i["message"]
                    k=gensim.utils.simple_preprocess(k)
                    tmp=[]
                  #### stop word  
                    for i in k:
                        if i not in stop_word:
                            tmp.append(i)
                    k=tmp
                  #####   
                    k=' '.join(k)
                    k=ViTokenizer.tokenize(k)
                    X.append(k)
                    #k=get_label(i["categories"])
                    k=get_label1(path)
                    y.append(k)

an_ninh_05.json
an_ninh_06.json
an_ninh_07.json
an_ninh_trat_tu.json
bat_dong_san.json
bien_dao.json
bien_dao_08_2.json
buu_chinh_vien_thong.json
chinh_sach.json
chinh_sach_08_2.json
chinh_tri.json
cong_nghiep_04.json
cong_nghiep_05.json
cong_nghiep_06.json
cong_nghiep_07.json
cong_nghiep_08_2.json
dan_toc_04.json
dan_toc_05.json
dan_toc_06.json
dan_toc_07.json
dau_thau_03.json
dau_thau_04.json
dau_thau_05.json
dau_thau_06.json
doi_ngoai_04.json
doi_ngoai_05.json
doi_ngoai_06.json
doi_ngoai_07.json
do_thi.json
du_lich.json
giai_tri.json
giao_duc.json
giao_thong_van_tai_02.json
giao_thong_van_tai_03.json
giao_thong_van_tai_04.json
giao_thong_van_tai_05.json
giao_thong_van_tai_06.json
giao_thong_van_tai_07.json
giao_thong_van_tai_08.json
hai_quan.json
khoa_hoc_cong_nghe_04.json
khoa_hoc_cong_nghe_05.json
khoa_hoc_cong_nghe_06.json
khoa_hoc_cong_nghe_07.json
khoa_hoc_cong_nghe_2_02.json
khoa_hoc_cong_nghe_2_03.json
khoa_hoc_cong_nghe_2_04.json
khoa_hoc_cong_nghe_2_05.json
khoa_hoc_cong_ng

# Xử lý

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
tfidf_vect = TfidfVectorizer(analyzer='word')
tfidf_vect.fit(X)
X_tfidf =  tfidf_vect.transform(X)
print(X_tfidf)

  (0, 130571)	0.01212858640706749
  (0, 129966)	0.06352963634452284
  (0, 129657)	0.03245582813781044
  (0, 129111)	0.030591857818691906
  (0, 129094)	0.03553570156719029
  (0, 128856)	0.1215599394446097
  (0, 128718)	0.01721770284908251
  (0, 128542)	0.09341631800282427
  (0, 128173)	0.03562923271933869
  (0, 128058)	0.3268162551321311
  (0, 127669)	0.02124948492317796
  (0, 127217)	0.0250656501229759
  (0, 125833)	0.039604674558556104
  (0, 124343)	0.024692752339249804
  (0, 121654)	0.06804781753765245
  (0, 120978)	0.052501401494575516
  (0, 120073)	0.03250236008624541
  (0, 118838)	0.030780296611817772
  (0, 118336)	0.03287637188486282
  (0, 118142)	0.023579966457019125
  (0, 118092)	0.014719701500193458
  (0, 117967)	0.0446419563582457
  (0, 116874)	0.025289811835800927
  (0, 115326)	0.029757159924491842
  (0, 112423)	0.0543428099068687
  :	:
  (61884, 36218)	0.07220919494602022
  (61884, 34751)	0.05946484610407415
  (61884, 33744)	0.04494814059219356
  (61884, 31549)	0.0437475220

In [7]:
import sklearn
encoder = sklearn.preprocessing.LabelEncoder()
y = encoder.fit_transform(y)

encoder.classes_

array(['bat dong san', 'bien dao', 'buu chinh vien thong', 'chinh sach',
       'chinh tri', 'cong nghiep', 'du lich', 'giai tri', 'giao duc',
       'giao thong van tai', 'hai quan', 'khoa hoc cong nghe',
       'kiem toan', 'kinh doanh', 'kinh te', 'phap luat', 'phu nu',
       'quoc hoi', 'quoc phong', 'suc khoe', 'tai chinh',
       'tai nguyen moi truong', 'thanh nien', 'thanh tra', 'the thao',
       'thoi trang', 'thuong mai', 'van hoa', 'van nghe', 'xa hoi',
       'xay dung', 'y te'], dtype='<U21')

In [8]:
from imblearn.over_sampling import SMOTE
oversample=SMOTE()
X_tfidf, y=oversample.fit_resample(X_tfidf, y)

Using TensorFlow backend.


In [11]:
import collections

print(collections.Counter(y))

Counter({0: 2944, 1: 2944, 2: 2944, 3: 2944, 4: 2944, 5: 2944, 6: 2944, 7: 2944, 8: 2944, 9: 2944, 10: 2944, 11: 2944, 12: 2944, 13: 2944, 14: 2944, 15: 2944, 16: 2944, 17: 2944, 18: 2944, 19: 2944, 20: 2944, 21: 2944, 22: 2944, 23: 2944, 24: 2944, 25: 2944, 26: 2944, 27: 2944, 28: 2944, 30: 2944, 29: 2944, 31: 2944})


In [16]:
import sklearn
encoder = sklearn.preprocessing.LabelEncoder()
y = encoder.fit_transform(y)

encoder.classes_

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31],
      dtype=int64)

In [25]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
tfidf_vect_ngram = TfidfVectorizer(analyzer='word', max_features=130000, ngram_range=(2, 3))
tfidf_vect_ngram.fit(X)
X_tfidf_ngram =  tfidf_vect_ngram.transform(X)
print(X_tfidf_ngram)

  (0, 128744)	0.04167989122273196
  (0, 127689)	0.09132781194451856
  (0, 126935)	0.0336562011082162
  (0, 126930)	0.025972913746350336
  (0, 126858)	0.040892694086792795
  (0, 126857)	0.031607778130990985
  (0, 125752)	0.08782519844745275
  (0, 125740)	0.045011102727779286
  (0, 125650)	0.03522200551576742
  (0, 124254)	0.0427402375893184
  (0, 124252)	0.032993680119341486
  (0, 124247)	0.04013161809802881
  (0, 124246)	0.040455960300958986
  (0, 124245)	0.06606532509267492
  (0, 124244)	0.043750468998864304
  (0, 124047)	0.03621349681937087
  (0, 124024)	0.04189722926246487
  (0, 124023)	0.04127113927753113
  (0, 123579)	0.0410784117583869
  (0, 123271)	0.04212422988433965
  (0, 123191)	0.03388219697393655
  (0, 123188)	0.2470431982316362
  (0, 123082)	0.04408004613114927
  (0, 122473)	0.04224162983952237
  (0, 122459)	0.021937599768060284
  :	:
  (61884, 33468)	0.06825532836448525
  (61884, 33399)	0.06285431940194196
  (61884, 32141)	0.06975318669642842
  (61884, 32134)	0.0999286085

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer


# create a count vectorizer object 
count_vect = CountVectorizer(analyzer='word', token_pattern=r'\w{1,}')
count_vect.fit(X)

# transform the training and validation data using count vectorizer object
X_count = count_vect.transform(X)


In [10]:
print(X_count)

  (0, 1730)	3
  (0, 4409)	1
  (0, 5686)	2
  (0, 5723)	1
  (0, 7449)	2
  (0, 8123)	1
  (0, 8234)	1
  (0, 9086)	12
  (0, 9210)	1
  (0, 10751)	1
  (0, 11041)	4
  (0, 13145)	1
  (0, 13442)	1
  (0, 13579)	1
  (0, 14503)	1
  (0, 14896)	1
  (0, 15043)	1
  (0, 16180)	2
  (0, 17486)	2
  (0, 18508)	2
  (0, 19587)	1
  (0, 20607)	2
  (0, 22720)	2
  (0, 23778)	1
  (0, 24314)	2
  :	:
  (61884, 103923)	2
  (61884, 105076)	3
  (61884, 105195)	1
  (61884, 105295)	1
  (61884, 105661)	2
  (61884, 106452)	1
  (61884, 107184)	3
  (61884, 107556)	1
  (61884, 107715)	1
  (61884, 107802)	1
  (61884, 109629)	3
  (61884, 110659)	1
  (61884, 111041)	5
  (61884, 112464)	1
  (61884, 113462)	3
  (61884, 116541)	1
  (61884, 121944)	1
  (61884, 122531)	7
  (61884, 126959)	1
  (61884, 127506)	4
  (61884, 128718)	3
  (61884, 128966)	1
  (61884, 130026)	1
  (61884, 130877)	1
  (61884, 130936)	5


In [11]:
from sklearn.decomposition import TruncatedSVD
svd_ngram = TruncatedSVD(n_components=300, random_state=42)
svd_ngram.fit(X_tfidf_ngram)

X_tfidf_ngram_svd = svd_ngram.transform(X_tfidf_ngram)
print(X_tfidf_ngram)

  (0, 128744)	0.04167989122273196
  (0, 127689)	0.09132781194451856
  (0, 126935)	0.0336562011082162
  (0, 126930)	0.025972913746350336
  (0, 126858)	0.040892694086792795
  (0, 126857)	0.031607778130990985
  (0, 125752)	0.08782519844745275
  (0, 125740)	0.045011102727779286
  (0, 125650)	0.03522200551576742
  (0, 124254)	0.0427402375893184
  (0, 124252)	0.032993680119341486
  (0, 124247)	0.04013161809802881
  (0, 124246)	0.040455960300958986
  (0, 124245)	0.06606532509267492
  (0, 124244)	0.043750468998864304
  (0, 124047)	0.03621349681937087
  (0, 124024)	0.04189722926246487
  (0, 124023)	0.04127113927753113
  (0, 123579)	0.0410784117583869
  (0, 123271)	0.04212422988433965
  (0, 123191)	0.03388219697393655
  (0, 123188)	0.2470431982316362
  (0, 123082)	0.04408004613114927
  (0, 122473)	0.04224162983952237
  (0, 122459)	0.021937599768060284
  :	:
  (61884, 33468)	0.06825532836448525
  (61884, 33399)	0.06285431940194196
  (61884, 32141)	0.06975318669642842
  (61884, 32134)	0.0999286085

In [12]:
svd = TruncatedSVD(n_components=300, random_state=42)
svd.fit(X_tfidf_ngram)

X_tfidf_ngram_svd = svd.transform(X_tfidf_ngram)
#X_test_tfidf_svd = svd.transform(X_test_tfidf)
X_tfidf_ngram_svd.shape

(61885, 300)

In [13]:
import sklearn
encoder = sklearn.preprocessing.LabelEncoder()
y = encoder.fit_transform(y)

encoder.classes_

array(['bat dong san', 'bien dao', 'buu chinh vien thong', 'chinh sach',
       'chinh tri', 'cong nghiep', 'du lich', 'giai tri', 'giao duc',
       'giao thong van tai', 'hai quan', 'khoa hoc cong nghe',
       'kiem toan', 'kinh doanh', 'kinh te', 'phap luat', 'phu nu',
       'quoc hoi', 'quoc phong', 'suc khoe', 'tai chinh',
       'tai nguyen moi truong', 'thanh nien', 'thanh tra', 'the thao',
       'thoi trang', 'thuong mai', 'van hoa', 'van nghe', 'xa hoi',
       'xay dung', 'y te'], dtype='<U21')

# Mô hình

Naive Bayes

In [12]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split    
    
X_train, X_val, y_train, y_val = train_test_split(X_tfidf, y, test_size=0.3, random_state=42)
       
cl=MultinomialNB().fit(X_train, y_train)

train_predictions = cl.predict(X_train)
val_predictions = cl.predict(X_val)

print("Train accuracy: ", sklearn.metrics.accuracy_score(train_predictions, y_train))
print("Validation accuracy: ", sklearn.metrics.accuracy_score(val_predictions, y_val))

Train accuracy:  0.6618242474789597
Validation accuracy:  0.6235714538442486


In [15]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split    
    
X_train, X_val, y_train, y_val = train_test_split(X_tfidf_ngram, y, test_size=0.3, random_state=42)
       
cl=MultinomialNB().fit(X_train, y_train)

train_predictions = cl.predict(X_train)
val_predictions = cl.predict(X_val)

print("Train accuracy: ", sklearn.metrics.accuracy_score(train_predictions, y_train))
print("Validation accuracy: ", sklearn.metrics.accuracy_score(val_predictions, y_val))

Train accuracy:  0.7275791223250767
Validation accuracy:  0.6413336205967898


Logistic Regression

In [13]:
from sklearn.linear_model import LogisticRegression

X_train, X_val, y_train, y_val = train_test_split(X_tfidf, y, test_size=0.3, random_state=42)
       
cl=LogisticRegression().fit(X_train, y_train)

train_predictions = cl.predict(X_train)
val_predictions = cl.predict(X_val)

print("Train accuracy: ", sklearn.metrics.accuracy_score(train_predictions, y_train))
print("Validation accuracy: ", sklearn.metrics.accuracy_score(val_predictions, y_val))

C:\Users\ASUS ZenBook\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Train accuracy:  0.815892031238153
Validation accuracy:  0.7482574390545943


In [16]:
from sklearn.linear_model import LogisticRegression

X_train, X_val, y_train, y_val = train_test_split(X_tfidf_ngram, y, test_size=0.3, random_state=42)
       
cl=LogisticRegression().fit(X_train, y_train)

train_predictions = cl.predict(X_train)
val_predictions = cl.predict(X_val)

print("Train accuracy: ", sklearn.metrics.accuracy_score(train_predictions, y_train))
print("Validation accuracy: ", sklearn.metrics.accuracy_score(val_predictions, y_val))

C:\Users\ASUS ZenBook\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Train accuracy:  0.8371384380987558
Validation accuracy:  0.7020898416460196


SVM

In [22]:
from sklearn.svm import SVC   
    
X_train, X_val, y_train, y_val = train_test_split(X_tfidf_ngram, y, test_size=0.3, random_state=42)
       
cl=SVC().fit(X_train, y_train)

train_predictions = cl.predict(X_train)
val_predictions = cl.predict(X_val)

print("Train accuracy: ", sklearn.metrics.accuracy_score(train_predictions, y_train))
print("Validation accuracy: ", sklearn.metrics.accuracy_score(val_predictions, y_val))

Train accuracy:  0.9365405480274245
Validation accuracy:  0.709253474092427


In [18]:
from sklearn.svm import SVC   
from sklearn.model_selection import train_test_split      
X_train, X_val, y_train, y_val = train_test_split(X_tfidf, y, test_size=0.3, random_state=42)
       
cl=SVC().fit(X_train, y_train)

train_predictions = cl.predict(X_train)
val_predictions = cl.predict(X_val)

print("Train accuracy: ", sklearn.metrics.accuracy_score(train_predictions, y_train))
print("Validation accuracy: ", sklearn.metrics.accuracy_score(val_predictions, y_val))

Train accuracy:  0.9301690802941845
Validation accuracy:  0.8202597034992747
